# Matriz de distancia y API de Google Maps

Nombre: Tamara Morel Cuevas

Instituto de Estadística Universidad de Valparaíso

Ingeniería en Estadística

Correo: tamaramorelc@gmail.com

Nombre profesor guía: Harvey Rosas Quintero

Correo: hjrosasq@uv.cl



### Generar puntos georeferenciados aleatorios

In [ ]:
import random
import math
datos =list()
# genera puntos aleatorios dentro de un radio 
def randomGeoref (centro, radio):# se define una función para generar puntos aleatorios
    y0 = centro[0]; 
    # se define como y0 la latitud que sera el primer elemento  como "punto de origen"
    x0 = centro[1]; # se define como x0 la longitud que especiquemos como "punto de origen"
    rd = radio / 111300; #//unos 111300 metros en un grado
    
    u = random.random(); #genera valores aleatorios
    v = random.random(); #genera valores aleatorios

    w = rd * math.sqrt(u);# genera una distancia aleatoria no mayor que el radio
    t = 2 * math.pi * v;  # convierte 0 a 1 en un ángulo aleatorio en radianes.
    
    #calcular la posición desde el ángulo y la distancia
    x = w * math.cos(t);
    y = w * math.sin(t);
    
    #xp = x / math.cos(y0);
    
    nuevalat = y + y0; #ahora se suma la posición y con la latitud de origen 
    nuevalon = x + x0; # se suma la posición x con la longitud de origen
    #newlon2 = xp + x0;
    
    return (nuevalat, nuevalon);

In [ ]:
aleatorio=[0 for x in range(18)] # creo una lista de o con un rango de 18
for i in range(18): # hago un ciclo for para generar 18 puntos aleatorios
    aleatorio[i] =randomGeoref((-33.0175803,-71.5503599), 1000) # banco de chile Av libertad 6 norte  con un radio de 1000 metros
    print( aleatorio[i] )

## Conectar la APi de Google a Python

1. Se debe crear una cuenta en Google Cloud Platform link: https://acortar.link/7PuPo además de registrar una tarjeta de crédito.

2. Ingresar a la consola de API de Google crear un proyecto en la parte superior de la consola 

3. Una vez creado el proyecto ingresar al menú a la opción de Apls y Servicios y seleccionar credenciales, para generar la clave de API donde debe ingresar las restricciones que seran las API a utilizar (Ej: matriz de distancia, geocodificación, direcciones, etc).


La tabla de precios para cada consulta o cantidad de datos a utlizar se registran en el siguiente link https://cloud.google.com/maps-platform/pricing/sheet?hl=es-419, donde a su vez todos los meses se da un crédito de $200 dolares.


Instalar el siguiente paquete para python

pip install -U googlemaps

pip install google-cloud-storage

In [ ]:
import googlemaps
# algunas librerias que serán necesarias
import pandas as pd
import numpy as np



## Geocodificación de direcciones

In [ ]:
#Comando para geocoficicar una direccion Ejemplo: 
gmaps = googlemaps.Client(key='Aqui va la Clave de API') 
geocode_result = gmaps.geocode('Reloj de Flores, Viña del Mar')
geocode_result

In [ ]:

#defino una función de gocodificador que me recorra una lista con valores latitud y longitud
#ya que solamente arroja resultados en formato JSON se selecciona lo deseado a guardar 
loc=list()
geo=[]
def geocode_direc(loc):
    gmaps = googlemaps.Client(key='Aqui va la Clave de API')
    p=0
    n=23 
    for i in range(n+1): #creo un ciclo for para que recorra toda la lista 
        geocode_resultado = gmaps.geocode(loc[p]) # utilizo el comando de gmaps.geocode para obtener los datos 
        p=p+1
        #como el texto esta en JSON selecciono lo que quiero que me imprima 
        #dire = geocode_resultado[0]["formatted_address"] 
        lat = geocode_resultado[0]["geometry"]["location"]["lat"] 
        lon = geocode_resultado[0]["geometry"]["location"]["lng"]
        geo.append((lat,lon))
        print (geo)

## Geocodificación inversa de puntos georeferenciados a direcciones

In [ ]:
#Comando para encontrar la dirección de puntos saliendo toda la información 
# en formato JSON Ejemplo: 
gmaps = googlemaps.Client(key='Aqui va la Clave de API') 
geocode_reverse = gmaps.reverse_geocode(-33.018844301912196, -71.55512222556408) 

In [ ]:
#defino una función de geocodificación inversa con una lista de datos lat y long
loc= list()
data = {}
data['Direcciones'] = []
def geocode_revez(loc):
    gmaps = googlemaps.Client(key='Aqui va la clave de API')
    p=0
    n=17
    for i in range(n+1):  
        geocode_resultado = gmaps.reverse_geocode(loc[p]) # utilizo el comando de gmaps.geocode para obtener los datos 
        p=p+1
        #como el texto esta en JSON selecciono lo que quiero que me imprima 
        data['Direcciones'].append((geocode_resultado[0]["formatted_address"]))
        #lat= geocode_resultado[0]["geometry"]["location"]["lat"] 
        #lon= geocode_resultado[0]["geometry"]["location"]["lng"] 
        print (data['Direcciones'])

## Matriz de Distancia de puntos 

In [ ]:
#Pregunta para una distancia en base a dos direcciones ejemplo:
gmaps = googlemaps.Client(key='Aqui va la Clave de API') 
gmaps.distance_matrix ('Alvares 646, viña del mar, Chile','6 Norte 536, Viña del Mar, Valparaíso, Chile')

In [ ]:
#crear un dataframe para guardar los datos obtenidos en este caso en km
lis=range(0,42)
kilometros = pd.DataFrame(columns=lis)
for e in lis:  #la rellenamos de 0 para poder acceder a la posiciones 
    kilometros.loc[e] = 0

In [ ]:
# definir la función para datos de distancia en kilometros 
# d y t son los puntos que se quieren comparar para las distancias
def matriz(d,t):
    gmaps = googlemaps.Client(key='Aqui va la clave de API') #declaro la clave de API Google Maps
    for i in range(len(d)):
        for j in range(len(d)):
            if i==j: 
                    kilometros.loc[i].loc[j]=0
            else:
                # para la distancia se coloca el mode='transit' que es para el trafico
                # traffic_model='duration_in_traffic' el cual tiene los tiempos previstos en el trafico según promedios históricos 
                resultado = gmaps.distance_matrix (prueba[i],prueba[j],mode='transit', traffic_model='duration_in_traffic') #llamo a la función de matriz de distancia            
                km = resultado["rows"][0]["elements"][0]["distance"]["value"]
                km= km/1000
                km="{:.1f}".format(km)
                kilometros.loc[i].loc[j]=km
    print (kilometros)

Para el caso de matriz de distancia en minutos es la misma funcion anterior pero cambiando la posición del dato a guardar que en este caso sería minutos

In [ ]:
# d y t son la lista de distacia a generar
#tambien se debe crear un DataFrame para guardar los datos  

def matriz(d,t):
    gmaps = googlemaps.Client(key='Aqui va la clave de API') #declaro la clave de API Google Maps
    for i in range(len(d)):
        
        for j in range(len(d)):
            if i==j: 
                    distancia.loc[i].loc[j]=0
            else:
            
                resultado = gmaps.distance_matrix (prueba[i],prueba[j],mode='transit', traffic_model='duration_in_traffic') #llamo a la función de matriz de distancia            
                tiempo = resultado["rows"][0]["elements"][0]["duration"]["value"] # declaro como tiempo el resultado de la duración del viaje
                tiempo= tiempo/60
                tiempo="{:.0f}".format(tiempo)
                distancia.loc[i].loc[j]=tiempo
    print (distancia)